In [1]:
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
# load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
# from sparse label to categorical
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [4]:
# reshape and normalize input images
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [5]:
x_train.shape

(60000, 28, 28, 1)

In [6]:
# network parameters
input_shape = (image_size, image_size, 1)
batch_size = 128
kernel_size = 3
filters = 64
dropout = 0.3

In [8]:
# use functional API to build cnn layers
inputs = Input(shape=input_shape)

y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(inputs)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)

# image to vector before connecting to dense layer
y = Flatten()(y)
# dropout regularization
y = Dropout(dropout)(y)

outputs = Dense(num_labels, activation='softmax')(y)

In [9]:
# build the model by supplying inputs/outputs
model = Model(inputs=inputs, outputs=outputs)
# network model in text
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0     

In [10]:
# classifier loss, Adam optimizer, classifier accuracy
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [11]:
# train the model with input images and labels
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          epochs=20,
          batch_size=batch_size)

Epoch 1/20
469/469 [==============================] - 6s 10ms/step - loss: 0.2871 - accuracy: 0.9122 - val_loss: 0.0608 - val_accuracy: 0.9795
Epoch 2/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0732 - accuracy: 0.9769 - val_loss: 0.0365 - val_accuracy: 0.9880
Epoch 3/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0522 - accuracy: 0.9835 - val_loss: 0.0298 - val_accuracy: 0.9894
Epoch 4/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0430 - accuracy: 0.9863 - val_loss: 0.0270 - val_accuracy: 0.9904
Epoch 5/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0352 - accuracy: 0.9885 - val_loss: 0.0301 - val_accuracy: 0.9899
Epoch 6/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0313 - accuracy: 0.9901 - val_loss: 0.0247 - val_accuracy: 0.9922
Epoch 7/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0285 - accuracy: 0.9908 - val_loss: 0.0254 - val_accuracy: 0.9917
Epoch

In [12]:
# model accuracy on test dataset
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size,
                       verbose=0)

print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))


Test accuracy: 99.4%
